# Experiment 1, Model 3

#### Model Setup

Run models in the following order, using their output labels as features for the next model:

1. Multilabel Linguistic Classifier
2. Multiclass Person Name + Occupation Sequence Classifier
3. Multilabel Stereotype + Omission Document Classifier

Train the first model and then run it over the entire dataset.

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/experiment1/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)
    
***

**Table of Contents**

[I.](#i) Train the Stereotype + Omission Classifier

[II.](#ii) Predict Over All Data

Load programming resources:

In [36]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For classification
import scipy
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix#, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support

Define resources for the models:

In [2]:
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics

In [3]:
# Model 1:
ling_label_subset = ["B-Generalization", "I-Generalization", "B-Gendered-Role", "I-Gendered-Role", "B-Gendered-Pronoun", "I-Gendered-Pronoun"]
# Model 2:
pers_o_label_subset = ["B-Unknown", "I-Unknown", "B-Feminine", "I-Feminine", "B-Masculine", "I-Masculine", "B-Occupation", "I-Occupation"]
# Model 3:
so_label_subset = ["B-Stereotype", "I-Stereotype", "B-Omission", "I-Omission"]

In [4]:
ling_label_tags = {
    "Gendered-Pronoun": ["B-Gendered-Pronoun", "I-Gendered-Pronoun"], "Gendered-Role": ["B-Gendered-Role", "I-Gendered-Role"],"Generalization": ["B-Generalization", "I-Generalization"]
    }
pers_o_label_tags = {
    "Unknown": ["B-Unknown", "I-Unknown"], "Feminine": ["B-Feminine", "I-Feminine"], "Masculine": ["B-Masculine", "I-Masculine"],
     "Occupation": ["B-Occupation", "I-Occupation"]
    }
so_label_tags = {
    "Stereotype": ["B-Stereotype", "I-Stereotype"], "Omission": ["B-Omission", "I-Omission"]
             }

In [5]:
d = 100               # dimensions of word embeddings (should match utils1.py) for file names
target_labels = "so"  # for file names

<a id="i"></a>
## I. Train the Stereotype + Omission Classifier

### Preprocessing

Load the document classification model's input data:

In [6]:
train = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_train.csv".format(target_labels), index_col=0)
dev = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_validate.csv".format(target_labels), index_col=0)
test = pd.read_csv(config.docc_path+"model_input/"+"{}_splits_as_csv/aggregated_final_test.csv".format(target_labels), index_col=0)
df_exp = pd.concat([train, dev, test])
df_exp["label"] = df_exp["label"].fillna("{'None'}")
df_exp = df_exp.loc[~df_exp.description.isna()]
df_exp = utils.getColumnValuesAsLists(df_exp, "label")
# df_exp.head()

Load the Linguistic, Person Name, and Occupation features and associate description IDs to the data, creating one row per description ID:

In [7]:
features_filename = "crf_{a}_{t}_baseline_fastText{d}_predictions_ALLDATA.csv".format(a="arow", t="pers_o", d=d)
df_features = pd.read_csv(config.experiment1_output_path+features_filename, usecols=["sentence_id", "token_id", "pred_ling_tag", "tag_pers_o_predicted"])
df_features = utils.getColumnValuesAsLists(df_features, "pred_ling_tag")
df_features = df_features.rename(columns={"tag_pers_o_predicted":"pers_o_pred", "pred_ling_tag":"ling_pred"})
# Generalize person name and occupation tags to labels
for label,tags in pers_o_label_tags.items():
    for tag in tags:
        df_features["pers_o_pred"] = df_features["pers_o_pred"].replace(to_replace=tag, value=label)
# df_features.head()
# df_features.pers_o_pred.value_counts()  # Looks good

In [8]:
df_desc = pd.read_csv(config.agg_path+"descs_sents_tokens_anns.csv", usecols=["description_id", "sentence_id", "token_id"])
df_desc = df_desc.set_index("description_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "sentence_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "token_id")
df_desc_exploded = df_desc.explode(["sentence_id", "token_id"])
df_desc_exploded = df_desc_exploded.reset_index()
df_desc_exploded = df_desc_exploded.astype("int64")
# df_desc_exploded.head()

In [9]:
joined = df_features.join(df_desc_exploded.set_index(["sentence_id", "token_id"]), on=["sentence_id", "token_id"])
grouped = utils.implodeDataFrame(joined, ["description_id"]).reset_index()
# grouped.head()

Flatten the lists of values in the `ling_pred` and `pers_o_pred` columns and remove duplicates from the lists:

In [10]:
feature_col1 = "ling_pred"
feature_col2 = "pers_o_pred"

In [11]:
ling = utils1.flattenFeatureCol(grouped, feature_col1)
perso = utils1.flattenFeatureCol(grouped, feature_col2)

In [12]:
grouped.insert(len(grouped.columns), "doc_"+feature_col1, ling)
grouped.insert(len(grouped.columns), "doc_"+feature_col2, perso)
# grouped.head()

Join the Linguistic and Person-Name+Occupation feature columns to the document classification model data:

In [13]:
features = grouped[["description_id", "doc_"+feature_col1, "doc_"+feature_col2]]
join_on = "description_id"
df = df_exp.join(features.set_index(join_on), on=join_on)
df = df.loc[~df.description.isna()]
df.head()

,description_id,start_offset,end_offset,field,description,subset,label,doc_ling_pred,doc_pers_o_pred
4699,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",train,[Omission],[Gendered-Role],[Masculine]
8942,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,train,[None],[Gendered-Pronoun],[Unknown]
5440,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,train,[None],[],[Unknown]
3474,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,train,"[Omission, Stereotype]","[Generalization, Gendered-Role]","[Masculine, Occupation, Feminine, Unknown]"
4769,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,train,[Omission],[Gendered-Pronoun],[Masculine]


Vectorize the documents, and binarize the features and targets:

In [14]:
def binarizeMultilabelTrainColumn(df_col):
    mlb = MultiLabelBinarizer()
    binarized = mlb.fit_transform(df_col)
    return mlb, binarized

def binarizeMultilabelDevColumn(mlb, df_col):
    binarized = mlb.transform(df_col)
    return binarized

In [15]:
train_df = df.loc[df.subset == "train"]
dev_df = df.loc[df.subset == "dev"]
target_col = "label"
feat1_col = "doc_ling_pred"
feat2_col = "doc_pers_o_pred"

In [16]:
mlb_target, y_train = binarizeMultilabelTrainColumn(train_df["label"])
y_dev = binarizeMultilabelDevColumn(mlb_target, dev_df["label"])
print(y_train.shape, y_dev.shape)

(16397, 3) (5452, 3)


In [17]:
mlb_feat1, train_feat1 = binarizeMultilabelTrainColumn(train_df[feat1_col])
dev_feat1 = binarizeMultilabelDevColumn(mlb_feat1, dev_df[feat1_col])
mlb_feat2, train_feat2 = binarizeMultilabelTrainColumn(train_df[feat2_col])
dev_feat2 = binarizeMultilabelDevColumn(mlb_feat2, dev_df[feat2_col])
print(train_feat1.shape, dev_feat1.shape)
print(train_feat2.shape, dev_feat2.shape)

(16397, 3) (5452, 3)
(16397, 4) (5452, 4)


In [18]:
cvectorizer = CountVectorizer()
tfidf = TfidfTransformer()
train_docs = cvectorizer.fit_transform(train_df["description"])
dev_docs = cvectorizer.transform(dev_df["description"])
train_docs = tfidf.fit_transform(train_docs)
dev_docs = tfidf.transform(dev_docs)
print(train_docs.shape, dev_docs.shape)

(16397, 26960) (5452, 26960)


In [40]:
train_feats = scipy.sparse.csr_matrix(np.concatenate([train_feat1, train_feat2], axis=1))
dev_feats = scipy.sparse.csr_matrix(np.concatenate([dev_feat1, dev_feat2], axis=1))

Concatenate the documents and features, creating one scipy sparse matrix for the train data and another for the dev data:

In [41]:
X_train = scipy.sparse.hstack([train_docs, train_feats])
X_dev = scipy.sparse.hstack([dev_docs, dev_feats])
print(X_train.shape, X_dev.shape)

(16397, 26967) (5452, 26967)


### Train & Predict

In [42]:
a = "sgd-svm"

Build a pipeline:

In [46]:
doc_clf = Pipeline([
#     ("vect", CountVectorizer()),
#     ("tfidf", TfidfTransformer()),
    ("clf", OneVsRestClassifier(SGDClassifier(loss="hinge")))  # Support Vector Machines loss function
    ])

In [47]:
doc_clf.fit(X_train, y_train)
predicted_dev = doc_clf.predict(X_dev)

### Peformance

Calculate performance metrics for the Stochastic Gradient Descent classifier

In [48]:
print("Dev Test Accuracy:", np.mean(predicted_dev == y_dev))

Dev Test Accuracy: 0.9405722670579604


In [75]:
classes = doc_clf.classes_
print(classes)
original_classes = mlb_target.classes_
print(original_classes)
label_dict = dict(zip(original_classes, classes))

[0 1 2]
['None' 'Omission' 'Stereotype']


Create a [confusion matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#multilabel-confusion-matrix) of the results, where, for class *i*:
* Count of true negatives (TN) is at position *i*,0,0
* Count of false negatives (FN) is at position *i*,1,0
* Count of true positives (FP) is at position *i*,1,1
* Count of false positives (PF) is at position *i*,0,1

In [76]:
matrix = multilabel_confusion_matrix(y_dev, predicted_dev, labels=classes)

In [78]:
scores = utils.getPerformanceMetrics(y_dev, predicted_dev, dev_matrix, classes, original_classes, label_dict)
scores = scores.tail(2) # Remove row for 'None'
scores = scores.drop(columns="true_neg")  # Not accurate because considers 'None' a class
scores["labels"] = original_classes[1:]
scores

,labels,false_neg,true_pos,false_pos,precision,recall,f_1
1,Omission,337,467,101,0.822183,0.580846,0.680758
2,Stereotype,97,218,14,0.939655,0.692063,0.797075


Save the performance results:

In [79]:
dir_path = config.tokc_path+"/experiment1/output/"
scores.to_csv(dir_path+"docclf_{a}_{t}_baseline_performance.csv".format(a=a, t=target_labels))

Add the predicted labels to the dev data:

In [85]:
pred_dev_labels = mlb_target.inverse_transform(predicted_dev)
# pred_dev_labels[0]

Add the classifier's labels to the `aggregated_validate.csv` DataFrame of descriptions to facilitate error analysis:

In [90]:
dev_df = dev_df.rename(columns={"label":"manual_label"})
dev_df.insert(len(dev_df.columns), "{a}_label".format(a=a), pred_dev_labels)
dev_df.head()
# print(len(pred_dev_labels), dev_df.shape)

,description_id,start_offset,end_offset,field,description,subset,manual_label,doc_ling_pred,doc_pers_o_pred,sgd-svm_label
5523,5523,367,1965,Biographical / Historical,"Edward Bald Jamieson, from Shetland, was a gra...",dev,[Stereotype],"[Gendered-Pronoun, Generalization]","[Occupation, Unknown, Masculine]","(Omission, Stereotype)"
4719,4719,5650,5811,Biographical / Historical,This likely refers to an article of the same t...,dev,[Omission],[],[Unknown],"(Omission,)"
735,735,7735,7881,Biographical / Historical,John Baillie kept a collection of the prayers ...,dev,[None],[Gendered-Pronoun],[Unknown],"(None,)"
2183,2183,1072,1372,Biographical / Historical,Joseph W. Hills graduated with the degree of M...,dev,[None],[Gendered-Pronoun],[Unknown],()
2299,2299,546,3642,Biographical / Historical,This collection is composed simply of an invit...,dev,"[Omission, Stereotype]",[Gendered-Pronoun],"[Occupation, Unknown, Masculine]","(Omission,)"


Save this version of the data:

In [91]:
dev_df.to_csv(dir_path+"aggregated_final_validate_predictions_docclf_{a}_{t}.csv".format(a=a, t=target_labels))